In [1]:
from pygalfitm import PyGalfitm
from pygalfitm.VOs import splus
from pygalfitm.auxiliars import string_times_x, get_dims, get_exptime, unpack_file
from pygalfitm.psf import make_psf

import splusdata

import os

In [2]:
name = "ari_test"

ra = 52.430082897775755	
dec = -33.557141932864596
cut_size = 200
bands = ["I", "R", "G"]

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "outputs/"

In [3]:
conn = splusdata.connect("gustavo", "asdflkjh")

You have access to internal data


In [4]:
pyg = PyGalfitm() 

In [5]:
input_images = ""
psf_images = ""
filters = ""

for band in bands:
    conn.get_cut(ra, dec, 200, band, filepath=os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'))
    
    unpack_file(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits.fz'))
    make_psf(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'), outfile=os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits'))

    input_images += "," + os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits')
    psf_images += "," + os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits')

    filters += "," + str(band).lower()

input_images = input_images[1:]
psf_images = psf_images[1:]
filters = filters[1:]

In [22]:
import pandas as pd

df = pd.read_csv("pygalfitm/VOs/splus_idr4_zps.csv")

In [27]:
from astropy.io.fits import getheader

header = getheader(os.path.join(DATA_FOLDER, name + "_r.fits"))
field = header['OBJECT']

In [39]:
zps = ""

for band in bands:
    zps += "," + str(df[df['Field'] == field][f'ZP_{band.lower().replace("f", "J0")}'].values[0])

zps = zps[1:]

In [40]:
zps

'23.344,23.535,23.49'

In [41]:
pyg.set_base({
    "A": input_images,
    "A1": filters,
    "B": os.path.join(OUTPUT_FOLDER, name + "ss.fits"),
    "C": "none",
    "D": psf_images,
    "A2": "7625,6231,4770",
    "H": f"1   {cut_size}  1   {cut_size}",
    "I": f"{cut_size} {cut_size}",
    "J": zps,
    "K": "0.55 0.55"
})

In [42]:
pyg.print_base()

A) data/ari_test_i.fits,data/ari_test_r.fits,data/ari_test_g.fits # Input data image (FITS file)
A1) i,r,g                            # Nick names (band labels) 
A2) 7625,6231,4770                   # Effective wavelenghts
B) outputs/ari_testss.fits          # Output data image block
C) none                             # Sigma image name (made from data if blank or 'none')
D) data/psf_ari_test_i.fits,data/psf_ari_test_r.fits,data/psf_ari_test_g.fits # Input PSF image and (optional) diffusion kernel
E) 1                                # PSF fine sampling factor relative to data 
F) none                             # Bad pixel mask (FITS image or ASCII coord list)
G) none                             # File with parameter constraints (ASCII file) 
H) 1   200  1   200                 # Image region to fit (xmin xmax ymin ymax)
I) 200 200                          # Size of the convolution box (x y)
J) 23.344,23.535,23.49              # Magnitude photometric zeropoint
K) 0.55 0.55           

In [43]:
axis_ratios, effective_rs, position_angles, mags = splus.get_sersic_splus(conn, ra, dec, bands)

finished
finished
finished


In [44]:
pyg.set_component("sersic", "1", string_times_x(cut_size / 2, 3))
pyg.set_component("sersic", "2", string_times_x(cut_size / 2, 3))
pyg.set_component("sersic", "3", mags)
pyg.set_component("sersic", "4", effective_rs)
pyg.set_component("sersic", "5", string_times_x("4", 3))
pyg.set_component("sersic", "9", axis_ratios)
pyg.set_component("sersic", "10", position_angles)

pyg.print_component("sersic")

1) 100.0,100.0,100.0                   1    band       # Position x [pixel]
2) 100.0,100.0,100.0                   1    band       # Position y [pixel]
3) 14.022123,14.107952,14.922325       3    band       # Integrated magnitude
4) 25.440218,30.087149,24.733025       2    band       # R_e (effective radius) [pix]
5) 4,4,4                               2    band       # Sersic index n (de Vaucouleurs n=4)
9) 0.6433851,0.7046503,0.7636552       1    band       # Axis ratio (b/a)
10) 15.269562,23.058279,32.80171        1    band       # Position angle (PA) [deg: Up=0, Left=90]
Z) 0                                                   # Skip this model in output image? (yes=1, no=0)


In [45]:
pyg.activate_components()
pyg.activate_components("sersic")

TypeError: 'list' object is not callable

In [46]:
pyg.write_feedme()

In [47]:
pyg.run()